### Análise de série de vazões (Xingó): 

In [1]:
import pandas as pd
import numpy as np
import plotly as py

import sys
sys.path.insert(0, r'C:\Users\arist\OneDrive\Documentos\UFAL\PIBIC 24-25\lib_clebson\HydroComp')

from files.ons import Ons
from files.ana import Ana
from series.flow import Flow
from series.chuva import Chuva

### 1 - Leitura dos dados e gerando hidrogramas

In [ ]:
file = "ONS_daily_flow.csv"
dados = pd.read_csv(file, index_col=0, parse_dates=True)

#Convetendo obj pandas para Series Flow
dados = Flow(pd.DataFrame(dados["XINGO (178)"]))

#Gerando hidrograma
fig, data = dados.plot_hydrogram()
py.offline.plot(fig, filename='gráficos/hidrograma.html')

#Calculando picos anuais
peaks_max = dados.maximum(station="XINGO (178)")

#Gerando hidrograma destanco os valores de pico
fig, data = peaks_max.plot_hydrogram()
py.offline.plot(fig, filename='gráficos/histo_max.html')


'gráficos/histo_max.html'

### **2 - Distribuições de probabilidade**

#### 2.1 - Método da máxima verossimilhança

In [24]:
## Máxima Verossimilhança
peaks_max.mvs()
peaks_max.fit

(-7.198629121675559, 2920.017124192904, 2.4988189701706567)

In [28]:
#Distribuição cumulativa
estimador = 'mvs'
title = 'Xingó Nat'
type_function = 'cumulative'
fig, data = peaks_max.plot_distribution(title, estimador, type_function)
py.offline.plot(fig, filename='gráficos/distrib_anual_mvs.html')

'gráficos/distrib_anual_mvs.html'

In [13]:
#Distribuição PDF
estimador = 'mvs'
title = 'Manso Nat'
type_function = 'density'
fig, data = peaks_max.plot_distribution(title, estimador, type_function)
py.offline.plot(fig, filename='gráficos/distrib_D_anual_mvs.html')

'gráficos/distrib_D_anual_mvs.html'

In [29]:
#Estimando magnitude de evento
period_return = 10
mag = peaks_max.magnitude(period_return, estimador)
mag

11159.545166357424

#### 2.2 - Método dos momentos L

In [ ]:
# Momentos L (parâmetros da distribuição)
peaks_max.mml()
peaks_max.fit

[-0.17538648901325302, 5988.272392336814, 1874.2360398256817]

In [ ]:
#Distribuição cumulativa
estimador = 'mml'
title = 'Xingo Nat'
type_function = 'cumulative'
fig, data = peaks_max.plot_distribution(title, estimador, type_function)
py.offline.plot(fig, filename='gráficos/distrib_anual_mml.html')

'gráficos/distrib_anual_mml.html'

In [ ]:
#Distribuição PDF
estimador = 'mml'
title = 'Xingo Nat'
type_function = 'density'
fig, data = peaks_max.plot_distribution(title, estimador, type_function)
py.offline.plot(fig, filename='gráficos/distrib_d_anual_mml.html')

'gráficos/distrib_d_anual_mml.html'

In [ ]:
#Estimando magnitude
period_return = 15
mag = peaks_max.magnitude(period_return, estimador)
mag

12381.887509137974

#### **3 - Séries de durações parciais**

In [32]:
station = "XINGO (178)"

#'stationary' or 'events_by_year'
type_threshold = 'stationary' 

#'flood' or 'drought'
type_event = 'flood'

#'media', 'mediana' or 'autocorrelation' 
type_criterion = 'median'

#Peaks or percentil
value_threshold = 0.75

peaks_sdp = dados.parcial(station, type_threshold, type_event, type_criterion, value_threshold)
peaks_sdp.peaks.head()

,Duration,Start,End,peaks
1931-05-06,128,1931-01-04,1931-05-12,7527.0
1933-02-20,30,1933-01-25,1933-02-24,6476.0
1934-02-07,35,1934-01-09,1934-02-13,6994.0
1935-02-24,98,1935-02-04,1935-05-13,7217.0
1937-03-11,37,1937-02-06,1937-03-15,6382.0


In [33]:
fig, data = peaks_sdp.plot_hydrogram(title='SDP-Xingo')
py.offline.plot(fig, filename='gráficos/histo_sdp_by_year.html')

'gráficos/histo_sdp_by_year.html'